# Szükséges packagek

In [1]:
import json
import tweepy
import csv

import time
import datetime

import requests
from bs4 import BeautifulSoup

import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from IPython.core.display import display, HTML

# Függvények

In [2]:
def Twitter_scrapelés(keyword_1, keyword_2, maximum_number_of_tweets_to_be_extracted, api):
    
    #Twitter key-ek lementése:
    
    consumer_key = api["api key"]
    consumer_secret = api["api secret"]
    access_key = api["access token key"]
    access_secret = api["access token secret"]

    # API endpoint

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth)

    #Scrapelés

    list_tweets_html = []
    for tweet in tweepy.Cursor(api.search, q= keyword_1 + " " + keyword_2, rpp=100).items(maximum_number_of_tweets_to_be_extracted):
        url = "http://twitter.com/{}/status/{}".format(tweet.user.screen_name,tweet.id)
        resp = requests.get("https://publish.twitter.com/oembed",
                            params={"url":url})    
        html_source = json.loads(resp.content)['html']
        list_tweets_html.append(html_source)    
    
    #üzenet összerakása
    global message_text
    message_text = """<html>
      <body>
        <p>Hello,<br>
           Ez itt egy próba email.<br>
           Ezeket a Tweeteket scrapeltük le a megadott hashtag alapján."""

    for tweet in list_tweets_html:
        message_text = message_text + "\n" + "\n"
        message_text = message_text + tweet + "\n"

    ending = """   </p>
      </body>
    </html>""" 
    
    message_text += ending

In [19]:
def email_küldés(receiver_email, sender_email, keyword_1, keyword_2, message_text, password, date):

    message = MIMEMultipart("alternative")
    message["Subject"] = "Heti Scholarship Tweetek: " + keyword_1 + " és " + keyword_2 + " kulcsszavakról" + " " + date
    message["From"] = sender_email
    message["To"] = receiver_email


    html = message_text
    message_content = MIMEText(html, "html")
    message.attach(message_content)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

# API kódok & engedélyek

In [4]:
api_key = "Ipm8hm7eTxjxuoRELDoULBnS1"
api_secret = "enmMreaDHqHyJEzlsF94cbJ1ErKejfkiRxlUAmnp4dy0cn0Nn0"
access_token_key = "3290381358-kzXn7bF9DpGB7397SxQbFRCEtQ0rPtbgaDPAVOl"
access_token_secret = "uKmJlCASied6M08UgAa4hGBHFSKLvTM4sRDlIkqPFz2jr"

api_dict = {"api key": api_key, "api secret": api_secret, "access token key": access_token_key, "access token secret": access_token_secret}
print(api_dict)

{'api key': 'Ipm8hm7eTxjxuoRELDoULBnS1', 'api secret': 'enmMreaDHqHyJEzlsF94cbJ1ErKejfkiRxlUAmnp4dy0cn0Nn0', 'access token key': '3290381358-kzXn7bF9DpGB7397SxQbFRCEtQ0rPtbgaDPAVOl', 'access token secret': 'uKmJlCASied6M08UgAa4hGBHFSKLvTM4sRDlIkqPFz2jr'}


# Írd be a szavakat, amik szerint szeretnél scrapelni

In [5]:
első_kulcsszó = input('legyen benne a scrapelt mailekben ez a kulcsszó: ')
második_kulcsszó = input('és ez a kulcsszó is: ')
max_tweetek = int(input('ennyi tweetet scrapeljünk: '))
password = input('jelszó (küldő email)')

üzenet = ""

legyen benne a scrapelt mailekben ez a kulcsszó: kitten
és ez a kulcsszó is: star
ennyi tweetet scrapeljünk: 10
jelszó (küldő email)stratkom2019


# Próba scrapelés

date_long = (str(datetime.datetime.now()))

date = date_long[:16]

Twitter_scrapelés(első_kulcsszó, második_kulcsszó, max_tweetek, api_dict) 

email_küldés("szaszi.aron@gmail.com", "rajkstratkom@gmail.com", első_kulcsszó, második_kulcsszó, message_text, password, date)

last_done = datetime.datetime.now() 
while True:
    if last_done < (datetime.datetime.now() - datetime.timedelta(seconds=120)):
        pass
        date_long = (str(datetime.datetime.now()))
        date = date_long[:16]
        Twitter_scrapelés(első_kulcsszó, második_kulcsszó, max_tweetek, api_dict)
        email_küldés("szaszi.aron@gmail.com", "rajkstratkom@gmail.com", első_kulcsszó, második_kulcsszó, message_text, password, date)
        last_done = datetime.datetime.now()

    time.sleep(30)

# Heti scrapelés

In [ ]:
date_long = (str(datetime.datetime.now()))

date = date_long[:10]

Twitter_scrapelés(első_kulcsszó, második_kulcsszó, max_tweetek, api_dict) 

email_küldés("szaszi.aron@gmail.com", "rajkstratkom@gmail.com", első_kulcsszó, második_kulcsszó, message_text, password, date)

last_done = datetime.datetime.now() 
while True:
    if last_done < (datetime.datetime.now() - datetime.timedelta(seconds=604800)):
        pass
        date_long = (str(datetime.datetime.now()))
        date = date_long[:10]
        Twitter_scrapelés(első_kulcsszó, második_kulcsszó, max_tweetek, api_dict)
        email_küldés("szaszi.aron@gmail.com", "rajkstratkom@gmail.com", első_kulcsszó, második_kulcsszó, message_text, password, date)
        last_done = datetime.datetime.now()

    time.sleep(3660)